In [44]:
import os
import json
import random
import string
import cv2
import h5py
import pickle
import argparse
import numpy as np

from utils import load_image, print_progress_bar

In [45]:
ins_file = json.load(open('dataset/annotations/instances_train2017.json', 'r'))

In [19]:
cat_id, id_cat = {}, {}
count = 0
cat_to_id, id_to_cat = {}, {}
i_r = {}
for cat in ins_file['categories']:
    cat_id[cat['name']] = cat['id']
    id_cat[cat['id']] = cat['name']
    
    cat_to_id[cat['name']] = count
    id_to_cat[count] = cat['name']
    
    i_r[cat['id']] = count
    count += 1

In [21]:
img_cat = {}
for ann in ins_file['annotations']:
    if ann['image_id'] not in img_cat:
        img_cat[ann['image_id']] = [ann['category_id']]
    elif ann['category_id'] not in img_cat[ann['image_id']]:
        img_cat[ann['image_id']].append(ann['category_id'])

In [14]:
list(img_cat.items())[:5]

[(558840, [58, 44, 47, 1, 50, 67]),
 (200365, [58, 67, 2, 3, 47]),
 (495357, [18, 4, 31, 1]),
 (116061, [18, 3, 4, 6, 1, 31, 44]),
 (16164, [18, 70])]

In [23]:
img_hot = {}
for img_id, cats in img_cat.items():
    mul_hot = [0] * 80
    for c in cats:
        mul_hot[i_r[c]] = 1
    img_hot[img_id] = mul_hot

In [30]:
id_to_file = {}
for i in ins_file['images']:
    id_to_file[i['id']] = i['file_name']

In [49]:
dataset_size = 5000
imgs = list(img_hot.keys())
random.shuffle(imgs)
imgs = imgs[:dataset_size]

In [50]:
def encode_images():
    """ Store images in a numpy array """

    images = []

    # Initial call to print 0% progress
    print_progress_bar_counter = 0
    print_progress_bar(print_progress_bar_counter, dataset_size, prefix = 'Progress:', suffix = 'Complete', length = 50)

    for image in imgs:
        img_array = load_image(
            'dataset/train2017/{}'.format(id_to_file[image]),
            size=(100, 100),
            color=False
        )
        images.append(img_array)

        # Update Progress Bar
        print_progress_bar_counter += 1
        print_progress_bar(print_progress_bar_counter, dataset_size, prefix = 'Progress:', suffix = 'Complete', length = 50)
        
    return images

x = encode_images()

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [51]:
y = []

# Initial call to print 0% progress
print_progress_bar_counter = 0
print_progress_bar(print_progress_bar_counter, dataset_size, prefix = 'Progress:', suffix = 'Complete', length = 50)

for image in imgs:
    y.append(img_hot[image])
    
    # Update Progress Bar
    print_progress_bar_counter += 1
    print_progress_bar(print_progress_bar_counter, dataset_size, prefix = 'Progress:', suffix = 'Complete', length = 50)

y = np.array(y, dtype=np.int64)

Progress: |███████████████████████████████-------------------| 63.5% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [52]:
h5f = h5py.File('caps/data.h5', 'w')
h5f.create_dataset('x', data=x)
h5f.create_dataset('y', data=y)
h5f.close()

In [53]:
mapping = {'id_cat': id_to_cat, 'cat_id': cat_to_id}
path = 'caps/mapping.pickle'
pickle_out = open(path, 'wb')
pickle.dump(mapping, pickle_out)
pickle_out.close()